In [1]:
# !pip install -U lightautoml
# !pip install pandas==1.4.3

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv
/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


In [3]:
X_TRAIN = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
X_TEST = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
X_ORIG = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')

In [4]:
X_TRAIN = pd.concat([X_TRAIN, X_ORIG])

In [5]:
X_TRAIN['person_emp_length'] = X_TRAIN['person_emp_length'].fillna(X_TRAIN['person_emp_length'].median())
X_TRAIN['loan_int_rate'] = X_TRAIN['loan_int_rate'].fillna(X_TRAIN['loan_int_rate'].median())

In [6]:
X_TEST.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39098 non-null  int64  
 1   person_age                  39098 non-null  int64  
 2   person_income               39098 non-null  int64  
 3   person_home_ownership       39098 non-null  object 
 4   person_emp_length           39098 non-null  float64
 5   loan_intent                 39098 non-null  object 
 6   loan_grade                  39098 non-null  object 
 7   loan_amnt                   39098 non-null  int64  
 8   loan_int_rate               39098 non-null  float64
 9   loan_percent_income         39098 non-null  float64
 10  cb_person_default_on_file   39098 non-null  object 
 11  cb_person_cred_hist_length  39098 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.6+ MB


In [7]:
X_TRAIN.pop('id')
id = X_TEST.pop('id')
Y_TRAIN = X_TRAIN.pop('loan_status')

In [8]:
numerical = X_TRAIN.select_dtypes(['int64', 'float64']).columns.to_list()
categorical = X_TRAIN.select_dtypes(['object']).columns.to_list()

In [9]:
print('numerical ', numerical)
print('categorical', categorical)

numerical  ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']
categorical ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']


In [10]:
X_TRAIN_COPY = X_TRAIN.copy()
X_TEST_COPY = X_TEST.copy()

X_TRAIN_COPY2 = X_TRAIN.copy()
X_TEST_COPY2 = X_TEST.copy()
categorical = X_TRAIN.select_dtypes(['object']).columns.to_list()
for col in categorical:
    X_TRAIN_COPY2[col] = X_TRAIN_COPY2[col].astype('category')
    X_TEST_COPY2[col] = X_TEST_COPY2[col].astype('category')

In [11]:
X_TRAIN_COPY2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91226 entries, 0 to 32580
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   person_age                  91226 non-null  int64   
 1   person_income               91226 non-null  int64   
 2   person_home_ownership       91226 non-null  category
 3   person_emp_length           91226 non-null  float64 
 4   loan_intent                 91226 non-null  category
 5   loan_grade                  91226 non-null  category
 6   loan_amnt                   91226 non-null  int64   
 7   loan_int_rate               91226 non-null  float64 
 8   loan_percent_income         91226 non-null  float64 
 9   cb_person_default_on_file   91226 non-null  category
 10  cb_person_cred_hist_length  91226 non-null  int64   
dtypes: category(4), float64(3), int64(4)
memory usage: 5.9 MB


In [12]:
def factorizeCategoricalColumns(data):
    dataCopy = data.copy()
    categorical = data.select_dtypes(['object']).columns.to_list()
    for col in categorical:
        dataCopy[col], _ = dataCopy[col].factorize(sort=True)
    return dataCopy

X_TRAIN_COPY = factorizeCategoricalColumns(X_TRAIN_COPY)
X_TEST_COPY = factorizeCategoricalColumns(X_TEST_COPY)

In [13]:
# oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# X_oe = pd.DataFrame(oe.fit_transform(X[cat_cols]), columns = cat_cols).fillna(0).astype(int)

# test_oe = pd.DataFrame(oe.transform(test_data[cat_cols]), columns = cat_cols).fillna(0).astype(int)
# X_oe_concated = pd.concat([X_oe, X[num_cols]], axis= 1)
# test_oe_concated = pd.concat([test_oe, test_data[num_cols]], axis= 1)

In [14]:
for col in X_TRAIN.columns.to_list():
    X_TRAIN[col] = X_TRAIN[col].astype('string')
    X_TEST[col] = X_TEST[col].astype('string')

In [15]:
#X_TRAIN.shape

lgb_params= {
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': -1,
    'verbosity': -1,
    'n_estimators': 1500,
    'max_bin': 1024,
    'boosting_type': 'gbdt', #'dart'
    'colsample_bytree': 0.5673775386473462,        
    'eta': 0.05446876730023387,
    'reg_lambda': 10.787843597294561,
    'min_child_samples': 69,
    'random_state': 42,
    #'early_stopping_rounds': 150,
    'verbose':1,
    #'categorical_feature': cat_indices,
}

xgb_params= {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'max_depth': 7,
    'eta': 0.07964177396162775,
    'reg_lambda': 38.499443612904315,
    'subsample': 0.8778759317150353,
    'colsample_bytree': 0.6504220261795185,
    'random_state': 42,
    'verbosity':0,
    'enable_categorical': True,
    'min_child_weight': 5,
    #'tree_method': 'hist',
}
xgb_params1 = {
    'eval_metric': 'auc',
    'n_estimators': 5000,
    'learning_rate': 0.10999995057648126,
    'max_depth': 8,
    'subsample': 0.9999999999999999,
    'colsample_bytree': 0.49681888251576845,
    'reg_alpha': 8.919028743833996e-06,
    'reg_lambda': 9.999999999999998,
    'gamma': 0.09787383664032377,
    'min_child_weight': 1.0,
    'max_bin': 262143,
#     'tree_method': 'hist',
#     'device': 'cuda',
    'enable_categorical': True,
    #'early_stopping_rounds': 100,
}

cb_params_3 = {'task_type': "CPU",
'iterations': 891,
'depth': 7,
'learning_rate': 0.0448563477253477,
'l2_leaf_reg': 8.309884320742215,
'border_count': 161,
'random_strength': 5.297381835241815,
'bagging_temperature': 0.10498408144882451,
'scale_pos_weight': 2.3753101495123747,
'verbose': False}

gb_best_params = {'random_state': 42,
                  'learning_rate': 0.08556786152814144, 
                  'max_depth': 11, 'n_estimators': 1130, 
                  'max_leaf_nodes': 90, 
                  'min_samples_leaf': 0.0025834075077401774, 
                  'min_samples_split': 0.11732169002596993, 
                  'min_weight_fraction_leaf': 0.0014666658781746856, 
                  'subsample': 0.9514006946730825}

hgb_params = {
    'categorical_features': categorical,
    'l2_regularization': 3.2797348813056515,
    'learning_rate': 0.04290001508632896,
    'max_depth': 292,
    #'max_features': 0.9605841946578942,
    'max_iter': 2978,
    'max_leaf_nodes': 38,
    'min_samples_leaf': 254,
    'random_state': 42
}

lgb_dart_params = {
    'boosting_type': 'dart',
    'colsample_bytree': 0.46939540409784575,
    'learning_rate': 0.0877911638744417,
    'min_child_samples': 61,
    'min_child_weight': 0.7580677713575383,
    'n_estimators': 877,
    'n_jobs': -1,
    'num_leaves': 35,
    'random_state': 1859,
    'reg_alpha': 2.5638237507752457,
    'reg_lambda': 1.8975230326974724,
    'scale_pos_weight': 2.7023753131827455,
    'subsample': 0.8009452917870442,
    'verbose': -1
}

lgb_goss_params = {
    'boosting_type': 'goss',
    'colsample_bytree': 0.24954643970263424,
    'learning_rate': 0.07188725215845243,
    'max_bins': 5626,
    'min_child_samples': 22,
    'min_child_weight': 0.7179252102190649,
    'n_estimators': 1319,
    'n_jobs': -1,
    'num_leaves': 21,
    'random_state': 42,
    'reg_alpha': 6.047743594793198,
    'reg_lambda': 5.277073776908381,
    'scale_pos_weight': 5.885539605359165,
    'subsample': 0.16046944461130142,
    'verbose': -1
}

cb_params_4 ={'objective': 'Logloss',
                                   'eval_metric': "AUC",
                                   'verbose': 0,
                                   'random_state': 42,
                                   'n_estimators': 5000,
                                   'scale_pos_weight': 2.3753101495123747,
                                   'learning_rate': 0.08746007467982597,
                                   'l2_leaf_reg': 3.50212953568363,
                                   'bagging_temperature': 0.15909129439715475,
                                   'random_strength': 1.6930893184821436,
                                   'depth': 4, 
                                   'min_data_in_leaf': 203,
                                   'grow_policy': 'SymmetricTree',
                                   'boosting_type': 'Plain',
                                }

In [16]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [17]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
predictions = [] # Initialize an array to store predictions
i = 0 
cb_params = {
    'border_count': 233,
    'colsample_bylevel': 0.411551721028093,
    'depth': 8,
    'iterations': 1664,
    'l2_leaf_reg': 1.5162407257384525,
    'learning_rate': 0.01380027964891961,
    'min_child_samples': 218,
    'random_state': 1859,
    'random_strength': 0.4028418793015393,
    'scale_pos_weight': 1.7183429881588008,
    'subsample': 0.996166121986487,
    'verbose': False
}
cb_params_2 = {'objective': 'Logloss',
                                   'eval_metric': "AUC",
                                   'verbose': 0,
                                   'random_state': 42,
                                   'n_estimators': 5000,
                                   'scale_pos_weight': 1.7183429881588008,
                                   'learning_rate': 0.057695680893866355,
                                   'l2_leaf_reg': 8.718243448685213,
                                   'bagging_temperature': 15.939315101482947,
                                   'random_strength': 1.2951116775412417,
                                   'depth': 5,
                                   'min_data_in_leaf': 65 
                                }
            
oof_scores = []
oof_predictions = []
oof_yval = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN, Y_TRAIN)):
    X_train, X_val = X_TRAIN.iloc[train_index], X_TRAIN.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    model =  CatBoostClassifier( **cb_params_3 , cat_features=X_TRAIN.columns.to_list()) 
     
#         loss_function='Logloss',
#         eval_metric='AUC',
#         learning_rate=0.5,       # learning rate used for the gradient step
#         iterations=100,          # max number of trees 
#         depth=5,                  # Depth of the trees
#         random_strength=1,        # Randomness to use for scoring splits when tree is selected. (Avoid overfitting)
#         l2_leaf_reg=50,            # regularization
#         task_type='GPU',

#         random_seed=42,
#         verbose=False,
        
#         # HPO from winning insurance approach
#         leaf_estimation_method="Newton",
#         bootstrap_type="Bernoulli",
#         grow_policy="SymmetricTree",
#         early_stopping_rounds=1,
#         score_function='NewtonL2',
#         leaf_estimation_iterations=1,
#         subsample=.8,
# #         reg_lambda=0.5,
#         od_pval=0.1, cat_features=X_TRAIN.columns.to_list())# Or your chosen model
    model.fit(X_train, y_train)
    
    oof = model.predict_proba(X_val)
    print(len(oof[:,1]))
    oof_predictions.append(oof[:,1])
    oof_yval.append(y_val)
    from sklearn.metrics import roc_auc_score
    roc_auc = roc_auc_score(y_val, oof[:,1])
    print(f"ROC-AUC Score: {roc_auc:.4f}")
    oof_scores.append(roc_auc)
    print("iteration done ", i)
    predictions.append(model.predict_proba(X_TEST)[:,1]) 
    i = i +1         # Store predictions for the validation fold

9123
ROC-AUC Score: 0.9716
iteration done  0
9123
ROC-AUC Score: 0.9702
iteration done  1
9123
ROC-AUC Score: 0.9687
iteration done  2
9123
ROC-AUC Score: 0.9685
iteration done  3
9123
ROC-AUC Score: 0.9657
iteration done  4
9123
ROC-AUC Score: 0.9684
iteration done  5
9122
ROC-AUC Score: 0.9687
iteration done  6
9122
ROC-AUC Score: 0.9627
iteration done  7
9122
ROC-AUC Score: 0.9697
iteration done  8
9122
ROC-AUC Score: 0.9643
iteration done  9


In [18]:
oof_scores1 = []
i =0
lgb_predictions = []
oof_predictions1 = []
oof_yval1 = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    lgb = LGBMClassifier(**lgb_params)
    lgb.fit(X_train, y_train, categorical_feature= categorical)
    
    oof1 = lgb.predict_proba(X_val)
    from sklearn.metrics import roc_auc_score
    oof_predictions1.append(oof1[:,1])
    oof_yval1.append(y_val)
    lgb_predictions.append(lgb.predict_proba(X_TEST_COPY)[:,1]) 
    roc_auc1 = roc_auc_score(y_val, oof1[:,1])
    print(f"ROC-AUC Score lightgbm: {roc_auc1:.4f}")
    oof_scores1.append(roc_auc1)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold

ROC-AUC Score lightgbm: 0.9653
iteration done  0
ROC-AUC Score lightgbm: 0.9659
iteration done  1
ROC-AUC Score lightgbm: 0.9626
iteration done  2
ROC-AUC Score lightgbm: 0.9640
iteration done  3
ROC-AUC Score lightgbm: 0.9579
iteration done  4
ROC-AUC Score lightgbm: 0.9632
iteration done  5
ROC-AUC Score lightgbm: 0.9617
iteration done  6
ROC-AUC Score lightgbm: 0.9541
iteration done  7
ROC-AUC Score lightgbm: 0.9627
iteration done  8
ROC-AUC Score lightgbm: 0.9582
iteration done  9


In [19]:
X_TRAIN_COPY2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91226 entries, 0 to 32580
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   person_age                  91226 non-null  int64   
 1   person_income               91226 non-null  int64   
 2   person_home_ownership       91226 non-null  category
 3   person_emp_length           91226 non-null  float64 
 4   loan_intent                 91226 non-null  category
 5   loan_grade                  91226 non-null  category
 6   loan_amnt                   91226 non-null  int64   
 7   loan_int_rate               91226 non-null  float64 
 8   loan_percent_income         91226 non-null  float64 
 9   cb_person_default_on_file   91226 non-null  category
 10  cb_person_cred_hist_length  91226 non-null  int64   
dtypes: category(4), float64(3), int64(4)
memory usage: 5.9 MB


In [20]:
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier
oof_scores2 = []
i =0
gb_predictions = []
oof_predictions2 = []
oof_yval2 = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    gb =  GradientBoostingClassifier(**gb_best_params)
    gb.fit(X_train, y_train)
    oof2 = gb.predict_proba(X_val)
    from sklearn.metrics import roc_auc_score
    oof_predictions2.append(oof2[:,1])
    oof_yval2.append(y_val)
    gb_predictions.append(gb.predict_proba(X_TEST_COPY)[:,1]) 
    roc_auc2 = roc_auc_score(y_val, oof2[:,1])
    print(f"ROC-AUC Score lightgbm: {roc_auc2:.4f}")
    oof_scores2.append(roc_auc1)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold

ROC-AUC Score lightgbm: 0.9635
iteration done  0
ROC-AUC Score lightgbm: 0.9657
iteration done  1
ROC-AUC Score lightgbm: 0.9637
iteration done  2
ROC-AUC Score lightgbm: 0.9643
iteration done  3
ROC-AUC Score lightgbm: 0.9577
iteration done  4
ROC-AUC Score lightgbm: 0.9627
iteration done  5
ROC-AUC Score lightgbm: 0.9602
iteration done  6
ROC-AUC Score lightgbm: 0.9535
iteration done  7
ROC-AUC Score lightgbm: 0.9644
iteration done  8
ROC-AUC Score lightgbm: 0.9577
iteration done  9


In [21]:
from sklearn.ensemble import HistGradientBoostingClassifier
oof_scores3 = []
i =0
hgb_predictions = []
oof_predictions3 = []
oof_yval3 = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    hgb =  HistGradientBoostingClassifier(**hgb_params)
    hgb.fit(X_train, y_train)
    oof3 = hgb.predict_proba(X_val)
    from sklearn.metrics import roc_auc_score
    oof_predictions3.append(oof3[:,1])
    oof_yval3.append(y_val)
    hgb_predictions.append(hgb.predict_proba(X_TEST_COPY)[:,1]) 
    roc_auc3 = roc_auc_score(y_val, oof3[:,1])
    print(f"ROC-AUC Score: {roc_auc3:.4f}")
    oof_scores3.append(roc_auc3)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold

ROC-AUC Score: 0.9577
iteration done  0
ROC-AUC Score: 0.9621
iteration done  1
ROC-AUC Score: 0.9566
iteration done  2
ROC-AUC Score: 0.9562
iteration done  3
ROC-AUC Score: 0.9526
iteration done  4
ROC-AUC Score: 0.9566
iteration done  5
ROC-AUC Score: 0.9520
iteration done  6
ROC-AUC Score: 0.9469
iteration done  7
ROC-AUC Score: 0.9585
iteration done  8
ROC-AUC Score: 0.9531
iteration done  9


In [22]:
from lightgbm import LGBMClassifier
oof_scores4 = []
i =0
lgb_dart_predictions = []
oof_predictions4 = []
oof_yval4 = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    lgb_dart = LGBMClassifier(**lgb_dart_params)
    lgb_dart.fit(X_train, y_train, categorical_feature= categorical)
    
    oof4 = lgb_dart.predict_proba(X_val)
    from sklearn.metrics import roc_auc_score
    oof_predictions4.append(oof4[:,1])
    oof_yval4.append(y_val)
    lgb_dart_predictions.append(lgb_dart.predict_proba(X_TEST_COPY)[:,1]) 
    roc_auc4 = roc_auc_score(y_val, oof4[:,1])
    print(f"ROC-AUC Score lightgbm: {roc_auc4:.4f}")
    oof_scores4.append(roc_auc4)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold

ROC-AUC Score lightgbm: 0.9596
iteration done  0
ROC-AUC Score lightgbm: 0.9639
iteration done  1
ROC-AUC Score lightgbm: 0.9614
iteration done  2
ROC-AUC Score lightgbm: 0.9617
iteration done  3
ROC-AUC Score lightgbm: 0.9550
iteration done  4
ROC-AUC Score lightgbm: 0.9601
iteration done  5
ROC-AUC Score lightgbm: 0.9593
iteration done  6
ROC-AUC Score lightgbm: 0.9528
iteration done  7
ROC-AUC Score lightgbm: 0.9620
iteration done  8
ROC-AUC Score lightgbm: 0.9585
iteration done  9


In [23]:
from xgboost import XGBClassifier
oof_scores6 = []
i =0
xgb_predictions = []
oof_predictions6 = []
oof_yval6 = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    xgb = XGBClassifier(**xgb_params1)
    xgb.fit(X_train, y_train)
    
    oof6 = xgb.predict_proba(X_val)
    from sklearn.metrics import roc_auc_score
    oof_predictions6.append(oof6[:,1])
    oof_yval6.append(y_val)
    xgb_predictions.append(xgb.predict_proba(X_TEST_COPY)[:,1]) 
    roc_auc6 = roc_auc_score(y_val, oof6[:,1])
    print(f"ROC-AUC Score lightgbm: {roc_auc6:.4f}")
    oof_scores6.append(roc_auc6)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold

ROC-AUC Score lightgbm: 0.9642
iteration done  0
ROC-AUC Score lightgbm: 0.9663
iteration done  1
ROC-AUC Score lightgbm: 0.9638
iteration done  2
ROC-AUC Score lightgbm: 0.9656
iteration done  3
ROC-AUC Score lightgbm: 0.9583
iteration done  4
ROC-AUC Score lightgbm: 0.9629
iteration done  5
ROC-AUC Score lightgbm: 0.9624
iteration done  6
ROC-AUC Score lightgbm: 0.9543
iteration done  7
ROC-AUC Score lightgbm: 0.9638
iteration done  8
ROC-AUC Score lightgbm: 0.9589
iteration done  9


In [24]:
from lightgbm import LGBMClassifier
oof_scores5 = []
i =0
lgb_goss_predictions = []
oof_predictions5 = []
oof_yval5 = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN_COPY, Y_TRAIN)):
    X_train, X_val = X_TRAIN_COPY.iloc[train_index], X_TRAIN_COPY.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    lgb_goss = LGBMClassifier(**lgb_goss_params)
    lgb_goss.fit(X_train, y_train, categorical_feature= X_train.columns.to_list())
    
    oof5 = lgb_goss.predict_proba(X_val)
    from sklearn.metrics import roc_auc_score
    oof_predictions5.append(oof5[:,1])
    oof_yval5.append(y_val)
    lgb_goss_predictions.append(lgb_goss.predict_proba(X_TEST_COPY)[:,1]) 
    roc_auc5 = roc_auc_score(y_val, oof5[:,1])
    print(f"ROC-AUC Score lightgbm: {roc_auc5:.4f}")
    oof_scores5.append(roc_auc5)
    print("iteration done ", i) 
    i = i +1         # Store predictions for the validation fold

ROC-AUC Score lightgbm: 0.9406
iteration done  0
ROC-AUC Score lightgbm: 0.9438
iteration done  1
ROC-AUC Score lightgbm: 0.9397
iteration done  2
ROC-AUC Score lightgbm: 0.9433
iteration done  3
ROC-AUC Score lightgbm: 0.9417
iteration done  4
ROC-AUC Score lightgbm: 0.9423
iteration done  5
ROC-AUC Score lightgbm: 0.9395
iteration done  6
ROC-AUC Score lightgbm: 0.9350
iteration done  7
ROC-AUC Score lightgbm: 0.9486
iteration done  8
ROC-AUC Score lightgbm: 0.9393
iteration done  9


In [25]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

cat2_predictions = [] # Initialize an array to store predictions
i = 0 
oof_scores7 = []
oof_predictions7 = []
oof_yval7 = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN, Y_TRAIN)):
    X_train, X_val = X_TRAIN.iloc[train_index], X_TRAIN.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    model =  CatBoostClassifier( **cb_params_2 , cat_features=X_TRAIN.columns.to_list()) 
     
    model.fit(X_train, y_train)
    
    oof7 = model.predict_proba(X_val)
    print(len(oof7[:,1]))
    oof_predictions7.append(oof7[:,1])
    oof_yval7.append(y_val)
    from sklearn.metrics import roc_auc_score
    roc_auc7 = roc_auc_score(y_val, oof7[:,1])
    print(f"ROC-AUC Score: {roc_auc7:.4f}")
    oof_scores7.append(roc_auc7)
    print("iteration done ", i)
    cat2_predictions.append(model.predict_proba(X_TEST)[:,1]) 
    i = i +1         # Store predictions for the validation fold

9123
ROC-AUC Score: 0.9719
iteration done  0
9123
ROC-AUC Score: 0.9697
iteration done  1
9123
ROC-AUC Score: 0.9683
iteration done  2
9123
ROC-AUC Score: 0.9685
iteration done  3
9123
ROC-AUC Score: 0.9659
iteration done  4
9123
ROC-AUC Score: 0.9682
iteration done  5
9122
ROC-AUC Score: 0.9691
iteration done  6
9122
ROC-AUC Score: 0.9627
iteration done  7
9122
ROC-AUC Score: 0.9699
iteration done  8
9122
ROC-AUC Score: 0.9652
iteration done  9


In [26]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

cat3_predictions = [] # Initialize an array to store predictions
i = 0 
oof_scores8 = []
oof_predictions8 = []
oof_yval8 = []
for fold, (train_index, val_index) in enumerate(skf.split(X_TRAIN, Y_TRAIN)):
    X_train, X_val = X_TRAIN.iloc[train_index], X_TRAIN.iloc[val_index]
    y_train, y_val = Y_TRAIN.iloc[train_index], Y_TRAIN.iloc[val_index]
 
    model =  CatBoostClassifier( **cb_params_4 , cat_features=X_TRAIN.columns.to_list()) 
     
    model.fit(X_train, y_train)
    
    oof8 = model.predict_proba(X_val)
    print(len(oof8[:,1]))
    oof_predictions8.append(oof8[:,1])
    oof_yval8.append(y_val)
    from sklearn.metrics import roc_auc_score
    roc_auc8 = roc_auc_score(y_val, oof8[:,1])
    print(f"ROC-AUC Score: {roc_auc8:.4f}")
    oof_scores8.append(roc_auc8)
    print("iteration done ", i)
    cat3_predictions.append(model.predict_proba(X_TEST)[:,1]) 
    i = i +1         # Store predictions for the validation fold

9123
ROC-AUC Score: 0.9715
iteration done  0
9123
ROC-AUC Score: 0.9704
iteration done  1
9123
ROC-AUC Score: 0.9686
iteration done  2
9123
ROC-AUC Score: 0.9678
iteration done  3
9123
ROC-AUC Score: 0.9647
iteration done  4
9123
ROC-AUC Score: 0.9677
iteration done  5
9122
ROC-AUC Score: 0.9685
iteration done  6
9122
ROC-AUC Score: 0.9626
iteration done  7
9122
ROC-AUC Score: 0.9695
iteration done  8
9122
ROC-AUC Score: 0.9646
iteration done  9


In [27]:
a=[1,2,3,4]
print(a)

[1, 2, 3, 4]


In [28]:
# oof_probas = np.stack(oof_predictions)
# oof_avg_probas = np.mean(oof_probas, axis=0)

# oof_probas1 = np.stack(oof_predictions1)
# oof_avg_probas1 = np.mean(oof_probas1, axis=0)

In [29]:
def objective_function(weights, oof_preds1, oof_preds2, oof_preds3,oof_preds4, 
                       oof_preds5,oof_preds6,oof_preds7,oof_preds8, oof_preds9,y_true):
#def objective_function(weights, oof_preds1, oof_preds2, y_true):    
  """Calculates the evaluation metric (e.g., accuracy) for given weights."""
  scores = [] 
  for i in range(10):
    oof_pred1 = oof_preds1[i]
    oof_pred2 = oof_preds2[i]
    oof_pred3 = oof_preds3[i]
    oof_pred4 = oof_preds4[i]
    oof_pred5 = oof_preds5[i]
    oof_pred6 = oof_preds6[i]
    oof_pred7 = oof_preds7[i]
    oof_pred8 = oof_preds8[i]
    oof_pred9 = oof_preds9[i]
#     print(oof_pred1.shape)
#     print(y_true[i].shape)
    
    total = weights[0]*oof_pred1 + weights[1]*oof_pred2 + weights[2]*oof_pred3 +weights[3]*oof_pred4+weights[4]*oof_pred5+weights[5]*oof_pred6 + weights[6]*oof_pred7 + weights[7]*oof_pred8+ weights[8]*oof_pred9
#     print(total)
    roc_auc = roc_auc_score(y_true[i], total)
    #print(f"ROC-AUC Score lightgbm: {roc_auc:.4f}")
    scores.append(roc_auc )
  return np.mean(scores)



In [30]:
def hill_climbing(oof_preds1, oof_preds2, oof_preds3,oof_preds4,oof_preds5, oof_preds6, 
                  oof_preds7,oof_preds8,oof_preds9,y_true, iterations=1000, step_size=0.01):
#def hill_climbing(oof_preds1, oof_preds2, y_true, iterations=1000, step_size=0.01):
  """Performs hill climbing to find optimal weights."""
  best_weights = [0.2, 0.1, 0.1, 0.1,0.1,0.1,0.1, 0.1,0.1]  # Initialize with equal weights
  best_score = objective_function(best_weights, oof_preds1, oof_preds2,
                                  oof_preds3, oof_preds4,oof_preds5, oof_preds6,oof_preds7,
                                  oof_preds8,oof_preds9, y_true)
  i = 0
  for _ in range(iterations):
    # Generate random neighbor weights
    new_weights = [w + np.random.uniform(-step_size, step_size) for w in best_weights]
    # Ensure weights sum to 1
    new_weights = [w / sum(new_weights) for w in new_weights]  
    new_score = objective_function(new_weights, oof_preds1, 
                                   oof_preds2, 
                                   oof_preds3, 
                                   oof_preds4,
                                   oof_preds5,
                                   oof_preds6,
                                   oof_preds7,
                                   oof_preds8,
                                   oof_preds9,
                                   y_true)
    i = i+1
    if i%100==0:
        print(i)
        
    # Update if new weights are better
    if new_score > best_score:
      best_score = new_score
      best_weights = new_weights

  return best_weights, best_score

In [31]:
# Assuming 'oof_preds1' and 'oof_preds2' are your OOF predictions
# and 'Y_TRAIN['outcome']' is your true target variable
best_weights, best_score = hill_climbing(oof_predictions, oof_predictions1, 
                                         oof_predictions2,
                                         oof_predictions3,
                                         oof_predictions4,
                                         oof_predictions5,
                                         oof_predictions6,
                                         oof_predictions7,
                                         oof_predictions8,
                                         oof_yval)
print(f"Best weights: {best_weights}, Best score: {best_score}")

100
200
300
400
500
600
700
800
900
1000
Best weights: [0.24881562867047316, 0.11592113874280088, -0.04367396812979813, 0.06179086737600691, 0.03728049622602646, -0.0006576518262287325, 0.15196693550684964, 0.23106983003185602, 0.19748672340201373], Best score: 0.9693080887776239


In [32]:
# Ensemble predictions using the optimized weights
len(oof_yval)

10

In [33]:
print('oof score roc auc is  ',np.mean(oof_scores))
print('oof score for lgb  ',np.mean(oof_scores1))
all_probas = np.stack(predictions)
avg_probas = np.mean(all_probas, axis=0)

all_probas1 = np.stack(lgb_predictions)
avg_probas1 = np.mean(all_probas1, axis=0)

all_probas2 = np.stack(gb_predictions)
avg_probas2 = np.mean(all_probas2, axis=0)

all_probas3 = np.stack(hgb_predictions)
avg_probas3 = np.mean(all_probas3, axis=0)

all_probas4 = np.stack(lgb_dart_predictions)
avg_probas4 = np.mean(all_probas4, axis=0)

all_probas5 = np.stack(lgb_goss_predictions)
avg_probas5 = np.mean(all_probas5, axis=0)

all_probas6 = np.stack(xgb_predictions)
avg_probas6 = np.mean(all_probas6, axis=0)

all_probas7 = np.stack(cat2_predictions)
avg_probas7 = np.mean(all_probas7, axis=0)

all_probas8 = np.stack(cat3_predictions)
avg_probas8 = np.mean(all_probas8, axis=0)

total_probas =  best_weights[0] * avg_probas + best_weights[1] * avg_probas1 + best_weights[2]*avg_probas2 + best_weights[3]*avg_probas3+ best_weights[4]*avg_probas4+ best_weights[5]*avg_probas5 + best_weights[6]*avg_probas6 + best_weights[7]*avg_probas7+ best_weights[8]*avg_probas8

oof score roc auc is   0.967865018083492
oof score for lgb   0.9615612220185863


In [34]:
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import recall_score, precision_score, log_loss, f1_score
# Y1 = Y_test
# #final_pred = result1_acc
# score = accuracy_score(Y1, final_pred)
# print('accuracy score is ', score)
# print('Recall score is',recall_score(Y1, final_pred))
# print('Precision score is',precision_score(Y1, final_pred))
# print('log loss is ',log_loss(Y1, final_pred))
# print('f1 score is ',f1_score(Y1, final_pred))

In [35]:
probs = total_probas
Result = pd.DataFrame()
Result['id']=id
if len(probs.shape) >1:
    Result['loan_status'] = probs[:,1]
else :
    Result['loan_status'] = probs

In [36]:
Result.to_csv("submission.csv",index=False)

In [37]:
pd.__version__

'2.2.3'

In [38]:
import xgboost as xgb

xgb.__version__

'2.0.3'

In [39]:
a, b = 3, 4

In [40]:
a

3